In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("transformations_actions")

sc = SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/27 11:47:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=transformations_actions>

# RDD 생성
파이썬의 리스트를 사용해서 RDD 생성 - `parallelize`

In [2]:
foods = sc.parallelize([
    "짜장면", "짬뽕", "마라탕",
    "떡볶이", "쌀국수","짬뽕",
    "짜장면", "짜장면", "짬뽕",
    "마라탕", "라면", "라면",
    "우동", "쌀국수"
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수']

# Actions
- 미뤄왔던 Transformations 작업을 모두 실행하고, 그 결과를 반환하는 작업
- RDD 생성이 아닌, 실제 데이터를 출력하는 역할을 한다.

In [3]:
# 각 원소 별 개수 세기
foods.countByValue()

defaultdict(int,
            {'짜장면': 3,
             '짬뽕': 3,
             '마라탕': 2,
             '떡볶이': 1,
             '쌀국수': 2,
             '라면': 2,
             '우동': 1})

In [4]:
# 상위 n개 데이터 가져오기
foods.take(5)

['짜장면', '짬뽕', '마라탕', '떡볶이', '쌀국수']

In [5]:
# 전체 데이터 가져오기
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수']

In [6]:
# RDD 내의 데이터 개수
foods.count()

14

# Narrow Transformations
- 1:1 변환을 의미한다.
- 하나의 열을 조작하기 위해서 다른 열이나 파티션의 데이터를 사용하지 않는다.

In [7]:
# map(<task>)
#  RDD 내의 데이터를 하나씩 꺼내서 `<task>`가 적용된 새로운 RDD가 만들어진다.
simple_rdd = sc.parallelize([1, 2, 3])
simple_rdd

ParallelCollectionRDD[4] at readRDDFromFile at PythonRDD.scala:274

In [8]:
# task 함수에는 반드시 "return" 이 있어야 한다!!!!
simple_rdd2 = simple_rdd.map(lambda x : x * 2)
simple_rdd2

PythonRDD[5] at RDD at PythonRDD.scala:53

In [9]:
simple_rdd2.collect()

[2, 4, 6]

In [10]:
# flatMap(<task>)
#  - map 함수와 매우 흡사하나, flatMap함수는 map의 모든 결과를 1차원 배열 형식으로 평평하게(flat) 나타낸다.
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [11]:
moviesRdd = sc.parallelize(movies)
moviesRdd.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [12]:
# 공백문자 기준으로 split() task 적용
mapMovies = moviesRdd.map(lambda x : x.split())
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [13]:
flatMapMovies = moviesRdd.flatMap(lambda x : x.split())
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [14]:
# filter(<task>)
#  - `task`의 결과가 True인 데이터만 추출
filtered_movies = flatMapMovies.filter(lambda x : x != '매트릭스')
filtered_movies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

# Wide Transformations

- 데이터의 파티션 또는 노드가 바뀔 수 있는 변환
- 셔플링이 일어날 수 있기 때문에 느리다.

In [15]:
foods = sc.parallelize([
    "짜장면", "짬뽕", "마라탕",
    "떡볶이", "쌀국수","짬뽕",
    "짜장면", "짜장면", "짬뽕",
    "마라탕", "라면", "라면",
    "우동", "쌀국수", "짬뽕밥", "짬짜면", "볶음밥", "볶짬면"
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수',
 '짬뽕밥',
 '짬짜면',
 '볶음밥',
 '볶짬면']

In [16]:
# groupBy(<task>)
#  groupBy의 task는 데이터를 묶어줄 수 있는 기준을 설정

# 그룹핑의 기준을 각 원소의 첫 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup.collect()
# 액션을 취햇기 때문에 더 이상 RDD 가 아니다 파이썬으로 생각해야됨!

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f5288ebaf40>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f5288eaab50>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f5288eaa730>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f5289745940>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f5289745400>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f5288f232e0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f5288f234c0>),
 ('볶', <pyspark.resultiterable.ResultIterable at 0x7f5288f238e0>)]

In [17]:
for k , v in foodsGroup.collect():
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕', '짬뽕', '짬뽕밥', '짬짜면']
마 ['마라탕', '마라탕']
떡 ['떡볶이']
쌀 ['쌀국수', '쌀국수']
라 ['라면', '라면']
우 ['우동']
볶 ['볶음밥', '볶짬면']


In [18]:
sc. stop()